In [85]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [86]:
Tsherpa_url= "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"


In [87]:
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')#크롬드라이버 창크기
options.add_argument("headless") #창 숨기기
options.add_argument('disable-gpu') #그래픽 성능 낮춰서 크롤링 성능 높아기
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome(options=options)
driver.get(Tsherpa_url)

In [88]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mainForm"]'))
    )
finally:
    pass

In [89]:
brk = 0
nxt = 2
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 0
stop_code = True

while True:
    
    page_count += 1
    brk += 1

    for i in range(1,11):
        
        time.sleep(0.5)
        hour = driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a/span').text
        
        if driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').is_displayed():
            driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').send_keys(Keys.ENTER)
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])

            nm = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[1]/span/a')
            yr = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[2]')
            scp = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]/img')[0].get_attribute('alt')
            desc = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td')
            print(scp)

        year = yr[0].text.split('|')[1].strip()
        year = year.split('.')[0].strip()
        
        if int(year) <= 2017:
            print('stop')
            stop_code = False
            break
        
        res.append([nm[0].text, hour, yr[0].text.split('|')[1].strip(), scp, desc[0].text])

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    if stop_code != True:
        print(brk)
        driver.quit()
        break


    # 다음 페이지로 넘어가는 코드입니다    
    if page_count != 11:
        driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[{page_count+nxt}]').send_keys(Keys.ENTER)

    else:
       driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[12]').send_keys(Keys.ENTER)
       page_count = 0

    time.sleep(2.5)



            





5점
5점
5점
5점
5점
4점
5점
5점
5점


KeyboardInterrupt: 

In [90]:
df = pd.DataFrame(res)
df.to_csv('./res_Tsherpa3.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df

,0,1,2,3,4
0,수업을 살리는 체육 레시피,30시간,2023.10.24,5점,아이들과 함께 할 체육활동을 이번 연수를 통해 좀 더 다양한 방법에 대해 알 수 있...
1,파워포인트로 끝내는 교실 콘텐츠 창작 비법 노트,15시간,2023.10.24,5점,"이번 연수를 통하여 매일 인디에서 다운받아서 쓰던 학급이름표, 보드게임 형식 등을 ..."
2,교육과정을 뒤집다! 백워드 통합단원 설계,15시간,2023.10.24,5점,백워드에 대한 막연한 개념만 있었는데\n강좌를 통하여 지식과 이해에 대한 구체적인 ...
3,파워포인트로 끝내는 교실 콘텐츠 창작 비법 노트,15시간,2023.10.23,5점,파워포인트로 이렇게 많은 것을 할 수 있다는 것을 새롭게 알게 되어서 많이 부끄럽습...
4,손그림과 파워포인트로 교실 속 콘텐츠 만들기,30시간,2023.10.23,5점,파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~\n감사합니다~
5,쉽게 배우고 활용하는 교실 속 손그림 그리기,15시간,2023.10.22,4점,도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..\n그런데 ...
6,왕초보를 위한 캘리그래피,15시간,2023.10.19,5점,저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...
7,같이 읽고 함께 나누는 독서수업방법,30시간,2023.10.19,5점,교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...
8,학생들과 함께 나누는 기후변화 이야기,30시간,2023.10.18,5점,"근래 들어 정말 기분좋게, 즐겁게 들었던 연수입니다.\n환경인가? 하며 들었는데 다..."


FileNotFoundError: [Errno 2] No such file or directory: 'res_Tsherpa4.csv'